In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [39]:
class scrap():
    def connect(self,url):#connect with the website
        response = requests.get(url)
        
        return BeautifulSoup(response.content,"html.parser")
    
    def table(self,soup): #extracting table
        table = soup.find_all("table")
        table = table[0]
        
        
        Sr_no=[]
        State_or_UT=[]
        Total_cases=[]
        Discharged=[]
        Deaths=[]

        rows = table.find_all("tr")[1:34]
        for row in rows:

            col = row.find_all("td")
            Sr_no.append(col[0].text.strip())
            State_or_UT.append(col[1].text.strip())
            Total_cases.append(col[2].text.strip())
            Discharged.append(col[3].text.strip())
            Deaths.append(col[4].text.strip())

            
        data = pd.DataFrame(list(zip(Sr_no,State_or_UT,Total_cases,Discharged,Deaths)),columns=["Sr.no","State/UT","Total_cases","Discharged","Deaths"])
        data=data.set_index("Sr.no")
        
        return data
    
    def clean(self,value):
        
        value.replace(r'^s*$',np.nan,regex=True,inplace=True)
        value.replace(np.nan,0,inplace=True)
        
        newcol=[]
        for i in value:
                
                s1 = str(i)
                s2 = s1.replace("+","")
                s3 = s2.replace(",","")

                s4 = int(s3)
                   
                newcol.append(s4)
        return newcol

In [40]:
s = scrap()
soup = s.connect("https://www.mohfw.gov.in/")
data = s.table(soup)


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 1 to 33
Data columns (total 4 columns):
State/UT       33 non-null object
Total_cases    33 non-null object
Discharged     33 non-null object
Deaths         33 non-null object
dtypes: object(4)
memory usage: 1.3+ KB


In [42]:
data["Total_cases"] = s.clean(data["Total_cases"])

data["Discharged"] = s.clean(data["Discharged"])

data["Deaths"] = s.clean(data["Deaths"])

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 1 to 33
Data columns (total 4 columns):
State/UT       33 non-null object
Total_cases    33 non-null int64
Discharged     33 non-null int64
Deaths         33 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.3+ KB


In [44]:
TCsum = data["Total_cases"].sum()
Dissum = data["Discharged"].sum()
Dsum = data["Deaths"].sum()

df = data.append({'State/UT':'Total','Total_cases':TCsum,'Discharged':Dissum,'Deaths':Dsum},ignore_index =True)

In [45]:
df

,State/UT,Total_cases,Discharged,Deaths
0,Andaman and Nicobar Islands,33,33,0
1,Andhra Pradesh,2307,1252,48
2,Arunachal Pradesh,1,1,0
3,Assam,90,41,2
4,Bihar,1018,438,7
5,Chandigarh,191,37,3
6,Chhattisgarh,66,56,0
7,Dadar Nagar Haveli,1,0,0
8,Delhi,8895,3518,123
9,Goa,15,7,0
